In [1]:
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine, inspect
import psycopg2 

In [2]:
pres_gdp = pd.read_csv("presidentfinal_df.csv")
pres_bday = pd.read_csv("birthdates-of-us-presidents.csv")

In [3]:
pres_bday=pres_bday.drop("csvbase_row_id",axis=1)
pres_bday = pres_bday.rename(columns={"Date of birth":"birthday","Name":"name"})

In [4]:
pres_gdp=pres_gdp.rename(columns={  'Year':'year',
                                    'President':'president',
                                    'Party':'party',
                                    'Nominal GDP (million of Dollars)':"nominal_gdp(million_of_dollars)",
                                    'Real GDP (millions of 2017 dollars)':'real_gdp(millions_of_2017_dollars)',
                                    'GDP Deflator (index 2017=100)':'gdp_deflator_(index_2017=100)',
                                    'Population':'population',
                                    'Nominal GDP per capita (current dollars)':'nominal_gdp_per_capita_(current_dollars)',
                                    'Real GDP per capita (year 2017 dollars)':'real_gdp_per_capita_(year_2017_dollars)'   
})

In [5]:
# PostgreSQL connection parameters
postgres_params = {
    "database": "postgres", # Insert the name of database
    "user": "postgres",     # Insert username if different
    "password": "",         # Insert your password
    "host": "localhost",    # Replace with your PostgreSQL host if not local
    "port": 5432,
    "options": "-c search_path=dbo,public"}

# Establish a connection to PostgreSQL
conn_str = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(**postgres_params)
postgres_engine = create_engine(conn_str)

# Write DataFrame to PostgreSQL
pres_gdp.to_sql('pres_gdp', con=postgres_engine, index=False, if_exists='replace')
pres_bday.to_sql('pres_bday', con=postgres_engine, index=False, if_exists='replace')

46

In [6]:
query="""
select * from pres_gdp;
"""
pd.read_sql(sql= query, con= postgres_engine)

,year,president,party,nominal_gdp(million_of_dollars),real_gdp(millions_of_2017_dollars),gdp_deflator_(index_2017=100),population,nominal_gdp_per_capita_(current_dollars),real_gdp_per_capita_(year_2017_dollars)
0,1789,George Washington,None,-3690167.12,-4284555.53,-17.10,-55758360.01,-12023.61,-11395.03
1,1790,George Washington,None,193,"4,975",3.88,"3,929,000",49.1,"1,266.23"
2,1791,George Washington,None,210,"5,274",3.98,"4,048,000",51.89,"1,302.85"
3,1792,George Washington,None,230,"5,663",4.05,"4,171,000",55.03,"1,357.63"
4,1793,George Washington,None,256,"6,113",4.19,"4,297,000",59.67,"1,422.52"
...,...,...,...,...,...,...,...,...,...
232,2019,Donald Trump,Republican,"21,521,395","20,692,100",104.01,"330,513,000","65,115.12","62,606.01"
233,2020,Donald Trump,Republican,"21,322,950","20,234,100",105.41,"331,800,000","64,266.79","60,985.03"
234,2021,Joe Biden,Democrat,"23,594,031","21,407,700",110.22,"332,367,000","70,991.30","64,412.92"
235,2022,Joe Biden,Democrat,"25,744,108","21,822,000",118.00,"333,568,000","77,171.74","65,414.65"
